In [1]:
import numpy as np
import pandas as pd
import scipy.optimize as opt

In [2]:
Geography = np.zeros(62,dtype = 'object')
income_mean = np.zeros((62,7))
income_median = np.zeros((62,7))
total_pop = np.zeros((62,7))
black_pop = np.zeros((62,7))
white_pop = np.zeros((62,7))
black_rate = np.zeros((62,7))
white_rate = np.zeros((62,7))

In [3]:
file_list = ['Income/ACS_11_5YR_S1901_with_ann.csv','Income/ACS_12_5YR_S1901_with_ann.csv','Income/ACS_13_5YR_S1901_with_ann.csv',
             'Income/ACS_14_5YR_S1901_with_ann.csv','Income/ACS_15_5YR_S1901_with_ann.csv','Income/ACS_16_5YR_S1901_with_ann.csv',
             'Income/ACS_17_5YR_S1901_with_ann.csv']
file_list2 = ['Population/ACS_11_5YR_B02001_with_ann.csv','Population/ACS_12_5YR_B02001_with_ann.csv','Population/ACS_13_5YR_B02001_with_ann.csv',
              'Population/ACS_14_5YR_B02001_with_ann.csv','Population/ACS_15_5YR_B02001_with_ann.csv','Population/ACS_16_5YR_B02001_with_ann.csv',
              'Population/ACS_17_5YR_B02001_with_ann.csv']
for i in range(6):
    data1 = pd.read_csv(file_list[i],header = None, skiprows = [0,1], names = ['zipcode','income median','income mean'], usecols = [2,91,99])
    data2 = pd.read_csv(file_list2[i],header = None, skiprows = [0,1], names = ['zipcode', 'total', 'white', 'black'], usecols = [2,3,5,7])
    for j in range(62):
        Geography[j] = data1['zipcode'][j]
        income_mean[j,i] = data1['income mean'][j]
        income_median[j,i] = data1['income median'][j]
        total_pop[j,i] = data2['total'][j]
        white_pop[j,i] = data2['white'][j]
        black_pop[j,i] = data2['black'][j]
        white_rate[j,i] = white_pop[j,i]/total_pop[j,i]
        black_rate[j,i] = black_pop[j,i]/total_pop[j,i]
data1 = pd.read_csv(file_list[6],header = None, skiprows = [0,1], names = ['zipcode','income median','income mean'], usecols = [2,25,27])
data2 = pd.read_csv(file_list2[6],header = None, skiprows = [0,1], names = ['zipcode', 'total', 'white', 'black'], usecols = [2,3,5,7])
for j in range(62):
    Geography[j] = data1['zipcode'][j]
    income_mean[j,6] = data1['income mean'][j]
    income_median[j,6] = data1['income median'][j]
    total_pop[j,6] = data2['total'][j]
    white_pop[j,6] = data2['white'][j]
    black_pop[j,6] = data2['black'][j]
    white_rate[j,6] = white_pop[j,6]/total_pop[j,6]
    black_rate[j,6] = black_pop[j,6]/total_pop[j,6]


In [4]:
Transition_init = 1/62*np.ones((62,62))
def Process(P):
    P_norm = np.zeros((62,62))
    for i in range(62):
        P_norm[i,:] = P[i,:]/P[i,:].sum()
    return black_pop[:,0].reshape(1,-1) @ P
def loss(P):
    Pre = Process(P.reshape((62,62)))
    return ((Pre-black_pop[:,1]*(black_pop[:,0].sum()/black_pop[:,1].sum()))**2).sum()

In [5]:
Transition_init.shape

(62, 62)

In [6]:
def loss(params):
    b1, b2, r = params
    Loss = []
    black_pre = np.zeros((62,7))
    black_pre[:,0] = black_pop[:,0]
    def pr(x1, x2, x3, x4):
        u1 = b1*np.log(x1) + b2*x2
        u2 = b1*np.log(x3) + b2*x4
        diff = u2-u1
        return 1/(1+np.exp(-diff))
    for i in range(6):
        probability = np.zeros((62,62))
        black_pop_end = np.zeros(62)
        black_rate_end_true = black_rate[:,i+1]
        income_start = income_mean[:,i]
        black_rate_start = black_rate[:,i]
        black_pop_start = black_pop[:,i]
        for j in range(62):
            for k in range(62):
                x1, x2, x3, x4 = income_start[j], black_rate_start[j], income_start[k], black_rate_start[k]
                probability[j,k] = pr(x1, x2, x3, x4)
        for q in range(62):
            probability[q,q] = 1
            total = probability[q,:].sum() - 1
            if total == 0:
                total = 1
            probability[q,:] = (1-r)*probability[q,:]/total
            probability[q,q] = r
        for p in range(62):
            black_pop_end[p] = (probability[:,p]*black_pop_start).sum()
        black_pre[:,i+1] = black_pop_end
        loss = (np.abs(black_pop_end-black_pop[:,i+1])).sum()
        Loss.append(loss)
    return np.array(Loss).sum()

In [7]:
params_init = Transition_init

results = opt.minimize(loss, params_init, bounds = [(1e-5, None)]*(62*62))
P = results.x.reshape((62,62))

print('P_MLE=', P)

ValueError: too many values to unpack (expected 3)

In [ ]:
results

In [ ]:
Process(P)

In [ ]:
def Mat(params):
    b1, b2, r = params
    Loss = []
    black_pre = np.zeros((62,7))
    black_pre[:,0] = black_pop[:,0]
    def pr(x1, x2, x3, x4):
        u1 = b1*x1 + b2*x2
        u2 = b1*x3 + b2*x4
        diff = u2-u1
        return 1/(1+np.exp(-diff))
    for i in range(6):
        probability = np.zeros((62,62))
        black_pop_end = np.zeros(62)
        black_pop_end_true = black_pop[:,i+1]
        income_start = income_mean[:,i]
        black_rate_start = black_rate[:,i]
        black_pop_start = black_pop[:,i]
        for j in range(62):
            for k in range(62):
                x1, x2, x3, x4 = income_start[j], black_rate_start[j], income_start[k], black_rate_start[k]
                probability[j,k] = pr(x1, x2, x3, x4)
        for q in range(62):
            probability[q,q] = 1
            total = probability[q,:].sum() - 1
            if total == 0:
                total = 1
            probability[q,:] = (1-r)*probability[q,:]/total
            probability[q,q] = r
        for p in range(62):
            List = list(range(62)).pop(p)
            black_pop_end[p] = (probability[:,p]*black_pop_start).sum()
        black_pre[:,i+1] = black_pop_end
        loss = ((black_pop_end - black_pop_end_true)**2).sum()
        Loss.append(loss)
    return black_pre

In [ ]:
np.abs(Mat([b1_MLE, b2_MLE, r_MLE])[black_pop!=0]/black_pop[black_pop!=0]-1).mean()

In [ ]:
np.array(Mat([b1_MLE, b2_MLE, r_MLE]),dtype='int')

In [ ]:
new_black_pop = np.zeros((62,7))
new_black_pop[:,0] = black_pop[:,0]
for i in range(6):
    new_black_pop[:,i+1] = black_pop[:,i]

In [ ]:
np.abs(new_black_pop[black_pop!=0]/black_pop[black_pop!=0]-1).mean()

In [8]:
def loss_0(params, i=1):
    b1, b2, r = params
    black_pre = np.zeros((62,7))
    black_pre[:,0] = black_pop[:,0]
    def pr(x1, x2, x3, x4):
        u1 = b1*np.log(x1) + b2*x2
        u2 = b1*np.log(x3) + b2*x4
        diff = u2-u1
        return 1/(1+np.exp(-diff))
    probability = np.zeros((62,62))
    black_pop_end = np.zeros(62)
    black_pop_end_true = black_pop[:,i+1]
    income_start = income_mean[:,i]
    black_rate_start = black_rate[:,i]
    black_pop_start = black_pop[:,i]
    for j in range(62):
        for k in range(62):
            x1, x2, x3, x4 = income_start[j], black_rate_start[j], income_start[k], black_rate_start[k]
            probability[j,k] = pr(x1, x2, x3, x4)
    for q in range(62):
        probability[q,q] = 1
        total = probability[q,:].sum() - 1
        if total == 0:
            total = 1
        probability[q,:] = (1-r)*probability[q,:]/total
        probability[q,q] = r
    for p in range(62):
        List = list(range(62)).pop(p)
        black_pop_end[p] = (probability[:,p]*black_pop_start).sum()
    black_pre[:,i+1] = black_pop_end
    loss = ((black_pop_end[black_pop_end_true!=0]/black_pop_end_true[black_pop_end_true!=0]-1)**2).sum()
    return loss

In [33]:
b1 = 0.1
b2 = 1
r = 0.9
params_init = np.array([b1, b2, r])
results = opt.minimize(loss_0, params_init, tol = 1e-14, bounds = [(0.00001,None),(None,None),(0,0.99)])

In [34]:
params_init = results.x

In [11]:
def mat(params, i=1):
    b1, b2, r = params
    black_pre = np.zeros((62,7))
    black_pre[:,0] = black_pop[:,0]
    def pr(x1, x2, x3, x4):
        u1 = b1*np.log(x1) + b2*x2
        u2 = b1*np.log(x3) + b2*x4
        diff = u2-u1
        return 1/(1+np.exp(-diff))
    probability = np.zeros((62,62))
    black_pop_end = np.zeros(62)
    black_pop_end_true = black_pop[:,i+1]
    income_start = income_mean[:,i]
    black_rate_start = black_rate[:,i]
    black_pop_start = black_pop[:,i]
    for j in range(62):
        for k in range(62):
            x1, x2, x3, x4 = income_start[j], black_rate_start[j], income_start[k], black_rate_start[k]
            probability[j,k] = pr(x1, x2, x3, x4)
    for q in range(62):
        probability[q,q] = 1
        total = probability[q,:].sum() - 1
        if total == 0:
            total = 1
        probability[q,:] = (1-r)*probability[q,:]/total
        probability[q,q] = r
    for p in range(62):
        black_pop_end[p] = (probability[:,p]*black_pop_start).sum()
    return black_pop_end

In [30]:
results

      fun: 1.1602352828821496
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.57593938e-03,  2.22044605e-08, -7.65032482e-04])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 208
      nit: 28
   status: 0
  success: True
        x: array([1.00000000e-02, 8.11344387e+02, 9.77876584e-01])

In [31]:
np.array(mat(params_init),dtype='int')

array([  682,    38,    39,     0,  4895,    50,  4200, 12612, 17949,
        6267,   907, 22236,  3199,  2655, 25312, 12343, 45735,  2583,
       66960, 74594, 34786,  4664, 28702, 38607,  3342, 13782, 69197,
       27111,   640,   237,  1616,  3883,  1060, 38093, 36402,  2174,
       15834, 12067,  1849,  2440, 36011, 44929,  7625,   453,  5957,
       41944, 41244, 21141, 29130,   557,  1956,   225,  1811,  2486,
        7058,   555,    78,  2749,  3033,  3015,  4180, 25735])

In [32]:
np.abs(np.array(mat(params_init),dtype='int')[black_pop[:,1]!=0]/np.array(black_pop[:,1],dtype='int')[black_pop[:,1]!=0]-1).mean()

0.021479780970397343

In [35]:
np.abs(np.array(mat(params_init),dtype='int')[black_pop[:,1]!=0]/np.array(black_pop[:,1],dtype='int')[black_pop[:,1]!=0]-1).mean()

0.021480719199788948

In [ ]:
np.abs(np.array(black_pop[:,1],dtype='int')[black_pop[:,2]!=0]/np.array(black_pop[:,2],dtype='int')[black_pop[:,2]!=0]-1).mean()

In [ ]:
np.abs(np.array(mat(params_init),dtype='int')[black_pop[:,2]!=0]/np.array(black_pop[:,2],dtype='int')[black_pop[:,2]!=0]-1).mean()

In [ ]:
np.array(black_pop[:,0],dtype='int')

In [ ]:
np.array(black_pop[:,1],dtype='int')

In [ ]:
np.array(black_pop, dtype = 'int')

In [ ]:
np.array(black_pop, dtype = 'int').sum(axis = 0)

In [ ]:
np.log(100000)

In [ ]:
cfdewq